In [80]:
import json
import ast

In [10]:
json_file = open('yelp_dataset/business.json')

In [11]:
json_file

<_io.TextIOWrapper name='yelp_dataset/business.json' mode='r' encoding='UTF-8'>

In [21]:
import collections

In [117]:
def rec(curline,i):
     
    if curline['attributes'] != None:
            for i in list(curline['attributes'].keys()):
                paramvalue = attrs[i]
                paramvalue.add(curline['attributes'][i])
                attrs[i] = paramvalue

# x = 0
attrs = collections.defaultdict(set)
for line in open('yelp_dataset/business.json','r'):
    curline = json.loads(line)
    if curline['attributes'] != None:
#             if x < 20:
        for param in list(curline['attributes'].keys()):
            paramvalue = attrs[param]
            subparam = ast.literal_eval(curline['attributes'][param])
            if type(subparam) == dict or type(subparam) == type(collections.defaultdict(set)):
                if type(paramvalue) == set:
#                     print('yeet')
                    paramvalue = collections.defaultdict(set)
                if type(paramvalue) == type(collections.defaultdict(set)):
#                     print('yote')
                    for subparamname in list(subparam.keys()):
                        paramvalue[subparamname].add(subparam[subparamname])
                attrs[param] = paramvalue
            else:
#                     print('6eet')
                if not subparam == None:
                    paramvalue.add(curline['attributes'][param])
                    attrs[i] = paramvalue
#         x += 1

In [118]:
attrs

defaultdict(set,
            {'GoodForKids': {'False', 'True'},
             'RestaurantsReservations': {'False', 'True'},
             'GoodForMeal': defaultdict(set,
                         {'dessert': {False, True},
                          'latenight': {False, True},
                          'lunch': {False, True},
                          'dinner': {False, True},
                          'brunch': {False, True},
                          'breakfast': {False, True}}),
             'BusinessParking': defaultdict(set,
                         {'garage': {False, True},
                          'street': {False, True},
                          'validated': {False, True},
                          'lot': {False, True},
                          'valet': {False, True}}),
             'Caters': {'False', 'True'},
             'NoiseLevel': {"'average'",
              "'loud'",
              "'quiet'",
              "'very_loud'",
              "u'average'",
              "u'loud'",